In [1]:
#@title Install required libraries

!pip install matplotlib~=3.10.0 \
  tensorflow~=2.20.0

print('\n\nAll requirements successfully installed.')






All requirements successfully installed.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#@title Code - Load dependencies

# data
import numpy as np
import pandas as pd

# machine learning
from tensorflow import keras


# data visualization
import plotly.express as px

In [2]:
# @title
chicago_taxi_dataset = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/chicago_taxi_train.csv")

In [3]:
#@title Code - Read dataset

# Updates dataframe to use specific columns.
training_df = chicago_taxi_dataset.loc[:, ('TRIP_MILES', 'TRIP_SECONDS', 'FARE', 'COMPANY', 'PAYMENT_TYPE', 'TIP_RATE')]

print('Read dataset completed successfully.')
print('Total number of rows: {0}\n\n'.format(len(training_df.index)))
training_df.head(200)

Read dataset completed successfully.
Total number of rows: 31694




,TRIP_MILES,TRIP_SECONDS,FARE,COMPANY,PAYMENT_TYPE,TIP_RATE
0,2.57,2341,31.99,Flash Cab,Mobile,6.3
1,1.18,1074,9.75,Flash Cab,Credit Card,27.9
2,1.29,1173,10.25,Sun Taxi,Cash,0.0
3,3.70,3360,23.75,Choice Taxi Association,Cash,0.0
4,1.15,1044,10.00,Flash Cab,Cash,0.0
...,...,...,...,...,...,...
195,1.13,821,9.00,Blue Ribbon Taxi Association,Mobile,22.9
196,0.57,414,6.00,Flash Cab,Cash,0.0
197,1.22,886,9.00,City Service,Cash,0.0
198,1.68,1219,9.00,Sun Taxi,Mobile,23.0


In [5]:
print('Total number of rows: {0}\n\n'.format(len(training_df.index)))
training_df.describe(include='all')

Total number of rows: 31694




,TRIP_MILES,TRIP_SECONDS,FARE,COMPANY,PAYMENT_TYPE,TIP_RATE
count,31694.000000,31694.000000,31694.000000,31694,31694,31694.000000
unique,NaN,NaN,NaN,31,7,NaN
top,NaN,NaN,NaN,Flash Cab,Credit Card,NaN
freq,NaN,NaN,NaN,7887,14142,NaN
mean,8.289463,1319.796397,23.905210,NaN,NaN,12.965785
std,7.265672,928.932873,16.970022,NaN,NaN,15.517765
min,0.500000,60.000000,3.250000,NaN,NaN,0.000000
25%,1.720000,548.000000,9.000000,NaN,NaN,0.000000
50%,5.920000,1081.000000,18.750000,NaN,NaN,12.200000
75%,14.500000,1888.000000,38.750000,NaN,NaN,20.800000


In [6]:
training_df.corr(numeric_only=True)

,TRIP_MILES,TRIP_SECONDS,FARE,TIP_RATE
TRIP_MILES,1.000000,0.800855,0.975344,-0.049594
TRIP_SECONDS,0.800855,1.000000,0.830292,-0.084294
FARE,0.975344,0.830292,1.000000,-0.070979
TIP_RATE,-0.049594,-0.084294,-0.070979,1.000000


In [7]:
#@title Code - View pairplot
px.scatter_matrix(training_df, dimensions=["FARE", "TRIP_MILES", "TRIP_SECONDS"])

In [4]:
class ExperimentSettings:
    def __init__(self,learning_rate,epochs,batch_size,input_features):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.input_features = input_features
def create_model(settings:ExperimentSettings, metrics:list[keras.metrics.Metric])->keras.Model:
    # One input tensor per feature (use a dict so Keras can accept feature-name keyed inputs)
    inputs = {name: keras.Input(shape=(1,), name=name) for name in settings.input_features}
    # Concatenate all input features into a single tensor
    concatenated_inputs = keras.layers.concatenate(list(inputs.values()))

    # Output: one neuron for regression
    outputs = keras.layers.Dense(units=1)(concatenated_inputs)

    model = keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=settings.learning_rate),
        loss="mean_squared_error",
        metrics=metrics
    )

    return model

def train_model(model:keras.Model,dataset:pd.DataFrame,label_name:str,settings:ExperimentSettings)->pd.DataFrame:
    """Train the model and return training history as a DataFrame."""
    # Ensure we pass numpy arrays for each feature so Keras gets consistent dtypes
    features = {name: dataset[name].values for name in settings.input_features}
    label = dataset[label_name].values

    history = model.fit(
        x=features,
        y=label,
        batch_size=settings.batch_size,
        epochs=settings.epochs,
        validation_split=0.2,
        verbose=1
    )

    # return training metrics history
    return pd.DataFrame(history.history)

print("✅ SUCCESS: Production-ready linear regression functions loaded.")

✅ SUCCESS: Production-ready linear regression functions loaded.


In [5]:
settings_1=ExperimentSettings(
    learning_rate=0.001,
    epochs=20,
    batch_size=50,
    input_features=['TRIP_MILES']
)

metrics=[keras.metrics.RootMeanSquaredError(name="rmse")]

model_1 = create_model(settings_1, metrics)
experiment_1 = train_model(model_1, training_df, 'FARE', settings_1)

#plot experiment metrics
# Plot experiment metrics: choose whichever rmse columns exist (robust to missing val_rmse)
metrics_cols = [c for c in ["rmse", "val_rmse"] if c in experiment_1.columns]
if not metrics_cols:
    # fallback: plot any column containing 'rmse' if present
    metrics_cols = [c for c in experiment_1.columns if 'rmse' in c.lower()]
fig_metrics = px.line(
    experiment_1[metrics_cols],
    labels={"index": "Epoch", "value": "RMSE"},
    title="Experiment 1: RMSE vs Epochs"
)
fig_metrics.show()
# Prepare features and predictions (pass a dict to match the model inputs)
features = {name: training_df[name].values for name in settings_1.input_features}
predictions_1 = model_1.predict(features).flatten()

fig_preds = px.scatter(
    x=training_df["TRIP_MILES"],
    y=training_df["FARE"],
    labels={"x": "TRIP_MILES", "y": "FARE"},
    title="Experiment 1: FARE vs TRIP_MILES with Predictions"
)
fig_preds.add_scatter(
    x=training_df["TRIP_MILES"],
    y=predictions_1,
    mode="markers",
    name="Predictions"
)
fig_preds.show()








Epoch 1/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 632.8693 - rmse: 25.1569 - val_loss: 1585.4923 - val_rmse: 39.8182
Epoch 2/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 632.8693 - rmse: 25.1569 - val_loss: 1585.4923 - val_rmse: 39.8182
Epoch 2/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 421.2440 - rmse: 20.5242 - val_loss: 953.7696 - val_rmse: 30.8832
Epoch 3/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 421.2440 - rmse: 20.5242 - val_loss: 953.7696 - val_rmse: 30.8832
Epoch 3/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 253.1665 - rmse: 15.9112 - val_loss: 485.0385 - val_rmse: 22.0236
Epoch 4/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 253.1665 - rmse: 15.9112 - val_loss: 485.0385 - val_rmse: 22.0236
Epoch 4/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 130.3835 - rmse: 11.4186 - val_loss: 178.2573 - val_rmse: 13.3513
Epoch 5/20
508/508 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 130.3835 - rmse: 11.4186 - val_loss: 178.2573 - val_rmse

991/991 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step
991/991 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step


In [8]:
#@title Code - Create train/test split (80/20)

np.random.seed(42)
num_rows = len(training_df)
perm = np.random.permutation(num_rows)
split_index = int(0.8 * num_rows)
train_idx = perm[:split_index]
test_idx = perm[split_index:]

train_df = training_df.iloc[train_idx].reset_index(drop=True)
test_df = training_df.iloc[test_idx].reset_index(drop=True)

print(f"Train rows: {len(train_df)}  |  Test rows: {len(test_df)}")


Train rows: 25355  |  Test rows: 6339


In [11]:
#@title Code - Validate Experiment 1 on test set (RMSE)
features_test_1 = {name: test_df[name].values for name in settings_1.input_features}
y_true_1 = test_df['FARE'].values
y_pred_1 = model_1.predict(features_test_1).flatten()
rmse_test_1 = float(np.sqrt(np.mean((y_true_1 - y_pred_1) ** 2)))
print(f"Experiment 1 Test RMSE: {rmse_test_1:.4f}")


199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step
Experiment 1 Test RMSE: 4.1315


In [12]:
#@title Code - Feature engineering: TRIP_MINUTES
# Derive minutes from seconds for both splits
train_df = train_df.copy()
test_df = test_df.copy()

train_df['TRIP_MINUTES'] = train_df['TRIP_SECONDS'] / 60.0
test_df['TRIP_MINUTES'] = test_df['TRIP_SECONDS'] / 60.0

print('Feature TRIP_MINUTES created for train and test sets.')


Feature TRIP_MINUTES created for train and test sets.


In [13]:
#@title Code - Experiment 2: Train with TRIP_MILES + TRIP_MINUTES and evaluate
settings_2 = ExperimentSettings(
    learning_rate=0.001,
    epochs=20,
    batch_size=50,
    input_features=['TRIP_MILES', 'TRIP_MINUTES']
)

model_2 = create_model(settings_2, metrics)
# Train on train split only
experiment_2 = train_model(model_2, train_df, 'FARE', settings_2)

# Plot metrics
metrics_cols_2 = [c for c in ["rmse", "val_rmse"] if c in experiment_2.columns]
if not metrics_cols_2:
    metrics_cols_2 = [c for c in experiment_2.columns if 'rmse' in c.lower()]
fig_metrics_2 = px.line(
    experiment_2[metrics_cols_2],
    labels={"index": "Epoch", "value": "RMSE"},
    title="Experiment 2: RMSE vs Epochs (Two Features)"
)
fig_metrics_2.show()

# Evaluate on test set
features_test_2 = {name: test_df[name].values for name in settings_2.input_features}
y_true_2 = test_df['FARE'].values
y_pred_2 = model_2.predict(features_test_2).flatten()
rmse_test_2 = float(np.sqrt(np.mean((y_true_2 - y_pred_2) ** 2)))
print(f"Experiment 2 Test RMSE: {rmse_test_2:.4f}")

# Quick comparison printout
print("\nComparison (lower is better):")
print(f"  Experiment 1 (miles only) Test RMSE: {rmse_test_1:.4f}")
print(f"  Experiment 2 (miles + minutes) Test RMSE: {rmse_test_2:.4f}")


Epoch 1/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1785.3640 - rmse: 36.9544 - val_loss: 1225.9901 - val_rmse: 35.0141
Epoch 2/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 755.6504 - rmse: 27.4891 - val_loss: 396.8873 - val_rmse: 19.9220
Epoch 3/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 179.9995 - rmse: 13.4164 - val_loss: 35.4451 - val_rmse: 5.9536
Epoch 4/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 17.2928 - rmse: 4.1585 - val_loss: 12.3960 - val_rmse: 3.5208
Epoch 5/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.3887 - rmse: 3.6591 - val_loss: 11.5638 - val_rmse: 3.4006
Epoch 6/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 12.8072 - rmse: 3.5787 - val_loss: 11.1839 - val_rmse: 3.3442
Epoch 7/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 12.4505 - rmse: 3.5285 - val_loss: 10.9792 - val_rmse: 3.3135
Epoch 8/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 12.2067 - rmse: 3.4938 - val_loss: 10.6151 - val_rmse: 3.2581
Epoch 9/20
4

199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step
Experiment 2 Test RMSE: 3.8449

Comparison (lower is better):
  Experiment 1 (miles only) Test RMSE: 4.1315
  Experiment 2 (miles + minutes) Test RMSE: 3.8449
